In [ ]:
import os, gc
import datetime
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing as pre
from sklearn.linear_model import LogisticRegression


path = os.path.join("C:\\Users\\Shawnr\\Documents\\Python scripts\\PBI and ML", "Online Retail.xlsx")
rd= pd.read_excel(path)

rd = rd[(rd.CustomerID.isnull()==False)&(rd.Description.isnull()==False)]
rd = rd[rd.Country=="United Kingdom"].reset_index(drop=True)

rd.drop(["Country", "Description"], axis=1, inplace=True)

rd.CustomerID = rd.CustomerID.astype(int)

rd["Amount"] =rd.Quantity*rd.UnitPrice
rd.drop(["Quantity", "UnitPrice"], axis=1, inplace=True)

rd["Date"] = rd.InvoiceDate.dt.date
rd["Time"] = rd.InvoiceDate.dt.time
rd.drop("InvoiceDate", axis=1, inplace=True)

now = rd.Date.max() + datetime.timedelta(days=1)
dt_by_cust = rd[["CustomerID", "Date"]].groupby("CustomerID")
recency = (now - dt_by_cust.max()).Date.apply(lambda d:d.days) 


freq = rd[["CustomerID", "InvoiceNo"]].groupby("CustomerID").apply(lambda x:len(x["InvoiceNo"].unique()))
mon = rd[["CustomerID", "Amount"]].groupby("CustomerID").sum()

rfm = pd.concat([recency, freq, mon], axis=1)
rfm.columns=["Recency", "Frequency", "Monetary"]

am_by_cust = rd[["CustomerID", "Amount"]].groupby("CustomerID")

rfm = pd.concat([recency, freq, mon], axis=1)
rfm.columns=["Recency", "Frequency", "Monetary"]

rfm = rfm[rfm.isnull()==False]

def generate_profile(rfm):
    rfm_profile = rfm.describe().transpose()
    rfm_profile["95%"] = rfm.quantile(0.95)
    rfm_profile["5%"] = rfm.quantile(0.05)
    return(rfm_profile)

rfm_profile = generate_profile(rfm)
for c in rfm_profile.index:
    rfm = rfm[rfm[c]>0]  
    rfm = rfm[rfm[c]<=rfm_profile.loc[c]["95%"]]
    rfm = rfm[rfm[c]>=rfm_profile.loc[c]["5%"]]

rfm["Monetary_Log"] = np.log(rfm.Monetary)
rfm["Recency_Log"] = np.log(rfm.Recency)
rfm["Frequency_Log"] = np.log(rfm.Frequency)

km = KMeans(n_clusters=4)
sd = pre.StandardScaler()
cols = ["Recency", "Frequency", "Monetary"]
rfm_st = sd.fit_transform(rfm[cols].values)
pca = PCA(n_components=2)
rfm_v = pca.fit_transform(rfm_st)
km.fit(rfm_st)

rfm["Cluster"] = ["Cluster_{}".format(l) for l in km.labels_]

rfm["pca_1"] = rfm_v[:,0]
rfm["pca_2"] = rfm_v[:,1]

rfm.reset_index(inplace=True)


dec_lr = LogisticRegression(C=0.8)
dec_lr.fit(X=rfm_st, y=rfm["Cluster"].values)

lr_coef = pd.DataFrame(data=dec_lr.coef_, 
                       columns=[["Recency", "Frequency", "Monetary"]])

lr_coef["Cluster"] = ["Cluster_0", "Cluster_1", "Cluster_2", "Cluster_3"]

In [3]:
lr_coef["Recency_Score"] = lr_coef.Recency.rank(ascending=False)
lr_coef["Frequency_Score"] = lr_coef.Frequency.rank(ascending=True)
lr_coef["Monetary_Score"] = lr_coef.Monetary.rank(ascending=True)
lr_coef["RFM_Score"] = lr_coef.Recency_Score + lr_coef.Frequency_Score + lr_coef.Monetary_Score
lr_coef["RFM_Score_Rank"] = lr_coef.RFM_Score.rank(ascending=True)
del km, sd, rfm_st, mon, freq, recency, dt_by_cust, rd
gc.collect()

201

In [4]:
lr_coef


,Recency,Frequency,Monetary,Cluster,Recency_Score,Frequency_Score,Monetary_Score,RFM_Score,RFM_Score_Rank
0,-0.855642,0.283825,0.314922,Cluster_0,3.0,3.0,3.0,9.0,3.0
1,-3.789509,-3.637469,-2.949861,Cluster_1,4.0,1.0,1.0,6.0,2.0
2,6.529839,-0.932483,-1.649654,Cluster_2,1.0,2.0,2.0,5.0,1.0
3,-0.032099,2.823334,2.798607,Cluster_3,2.0,4.0,4.0,10.0,4.0
